In [40]:
import math

In [41]:
def grouper(iterable, max_diff):
    prev = None
    group = []
    for item in iterable:
        if not prev or item - prev <= max_diff:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group

In [89]:
elong_times = [60,60, 46, 60, 45, 30, 200, 100]
elong_times.sort()
elong_time_max_diff = 10

In [90]:
groups = dict(enumerate(grouper(elong_times,elong_time_max_diff), 1))
groups

{1: [30], 2: [45, 46], 3: [60, 60, 60], 4: [100], 5: [200]}

In [92]:
grouper(elong_times,elong_time_max_diff)

<generator object grouper at 0x7fabab806c80>

In [93]:
from pydna.dseqrecord import Dseqrecord
from pydna.readers import read
from pydna.amplify import pcr
from pydna.primer import Primer
from Bio.SeqRecord import SeqRecord
    
# initialize
middle = 'a'*2000
template = Dseqrecord("tacactcaccgtctatcattatcagcgacgaagcgagcgcgaccgcgagcgcgagcgca"+middle+"caggagcgagacacggcgacgcgagcgagcgagcgatactatcgactgtatcatctgatagcac")
p1 = Primer("tacactcaccgtctatcattatc")
p2 = Primer("cgactgtatcatctgatagcac").reverse_complement()
amplicon = pcr(p1, p2, template)
amplicon.annotations['polymerase'] = "OneTaq Hot Start"
det_proc_speed(amplicon)


Amplicon(2123)